<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [1]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

from bs4 import BeautifulSoup

## 1) *Clean:* Job Listings from indeed.com that contain the title "Data Scientist" 

You have `job_listings.csv` in the data folder for this module. The text data in the description column is still messy - full of html tags. Use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to clean up this column. You will need to read through the documentation to accomplish this task. 

In [3]:
from bs4 import BeautifulSoup
import requests

##### Your Code Here #####
df = pd.read_csv('.\data\job_listings.csv')
df.head()                


,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


In [15]:
df['description2'] = [BeautifulSoup(text).get_text() for text in df['description']]
df.head()
print(df.shape)

(426, 5)


In [18]:
df['description2'][1]


'b\'Job Description\\n\\nAs a Data Scientist 1, you will help us build machine learning models, data pipelines, and micro-services to help our clients navigate their healthcare journey. You will do so by empowering and improving the next generation of Accolade Applications and user experiences.\\nA day in the life\\xe2\\x80\\xa6\\nWork with a small agile team to design and develop mobile applications in an iterative fashion.\\nWork with a tight-knit group of development team members in Seattle.\\nContribute to best practices and help guide the future of our applications.\\nOperates effectively as a collaborative member of the development team.\\nOperates effectively as an individual for quick turnaround of enhancements and fixes.\\nResponsible for meeting expectations and deliverables on time with high quality.\\nDrive and implement new features within our mobile applications.\\nPerform thorough manual testing and writing test cases that cover all areas.\\nIdentify new development tool

In [19]:
def clean_descriptions(desc):

    # transform unicode to ascii
    desc = (desc.
            replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xc2\\xa8', '').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")
           )
    # use BeautifulSoup to strip html tags
    soup = BeautifulSoup(desc)
    for st in soup(['script', 'style']):
        s.decompose()
    desc = ' '.join(soup.stripped_strings)
    
    # remove '\n' from string
    desc = (desc.
            replace('\\n', ' ').
            replace('\n',  ' ').
            replace('/', ' ').
            replace('$', '').
            replace(',', '')
           )
    
    #remove b' at begining of each string
    desc = desc[2:]
    
    # remove non aplh characters
    desc = re.sub(r'[^a-zA-Z ]', '', desc)
    
    desc = str.strip(desc)
    
    return desc

In [20]:
df['clean_description'] = df['description2'].apply(lambda x: clean_descriptions(x))
df.head()

,Unnamed: 0,description,title,description2,tokens,clean_description
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,"b""Job Requirements:\nConceptual understanding ...","[b""Job, Requirements:\nConceptual, understandi...",Job Requirements Conceptual understanding in M...
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"b'Job Description\n\nAs a Data Scientist 1, yo...","[b'Job, Description\n\nAs, a, Data, Scientist,...",Job Description As a Data Scientist you will...
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,b'As a Data Scientist you will be working on c...,"[b'As, a, Data, Scientist, you, will, be, work...",As a Data Scientist you will be working on con...
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"b'$4,969 - $6,756 a monthContractUnder the gen...","[b'$4,969, -, $6,756, a, monthContractUnder, t...",a monthContractUnder the general supervision o...
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,b'Location: USA \xe2\x80\x93 multiple location...,"[b'Location:, USA, \xe2\x80\x93, multiple, loc...",Location USA multiple locations years of Ana...


In [21]:
df['clean_description'][1]

'Job Description  As a Data Scientist  you will help us build machine learning models data pipelines and microservices to help our clients navigate their healthcare journey You will do so by empowering and improving the next generation of Accolade Applications and user experiences A day in the life Work with a small agile team to design and develop mobile applications in an iterative fashion Work with a tightknit group of development team members in Seattle Contribute to best practices and help guide the future of our applications Operates effectively as a collaborative member of the development team Operates effectively as an individual for quick turnaround of enhancements and fixes Responsible for meeting expectations and deliverables on time with high quality Drive and implement new features within our mobile applications Perform thorough manual testing and writing test cases that cover all areas Identify new development tools approaches that will increase code quality efficiency an

## 2) Use Spacy to tokenize the listings 

In [11]:
##### Your Code Here #####
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [22]:
tokens = []

for doc in tokenizer.pipe(df['clean_description'], batch_size=500):
    doc_tokens = [token.text for token in doc]
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

df.head()

,Unnamed: 0,description,title,description2,tokens,clean_description
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,"b""Job Requirements:\nConceptual understanding ...","[Job, Requirements, Conceptual, understanding,...",Job Requirements Conceptual understanding in M...
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"b'Job Description\n\nAs a Data Scientist 1, yo...","[Job, Description, , As, a, Data, Scientist, ...",Job Description As a Data Scientist you will...
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,b'As a Data Scientist you will be working on c...,"[As, a, Data, Scientist, you, will, be, workin...",As a Data Scientist you will be working on con...
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"b'$4,969 - $6,756 a monthContractUnder the gen...","[a, monthContractUnder, the, general, supervis...",a monthContractUnder the general supervision o...
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,b'Location: USA \xe2\x80\x93 multiple location...,"[Location, USA, , multiple, locations, , yea...",Location USA multiple locations years of Ana...


In [23]:
# add common words to stop words
STOP_WORDS = nlp.Defaults.stop_words.union([
    'job',
    'requirements',
    'description',
    'you',
    'strong',
    'data',
    'desire',
    'qualifications',
    ])

def tokenize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [24]:
df['tokens'] = df['clean_description'].apply(lambda x: tokenize(x))

df['tokens'].head()

0    [job, requirement, conceptual, understanding, ...
1    [job, description,  , Data, scientist,  , help...
2    [Data, scientist, work, consult, business, res...
3    [monthcontractunder, general, supervision, Pro...
4    [location, USA,  , multiple, location,  , year...
Name: tokens, dtype: object

## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## 4) Visualize the most common word counts

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 